In [0]:
%python
df=spark.read.table("dev.naval_bronze.sales")

In [0]:
%python
df_cleaned=df.dropDuplicates().dropna().drop("ingestion_date")

In [0]:
%python
df_cleaned.write.mode("overwrite").saveAsTable("dev.naval_silver.sales_cleaned")

In [0]:
select * from dev.naval_bronze.products

In [0]:
create or replace table dev.naval_silver.product_scd (product_id int, product_name string, product_category string, product_price double)

In [0]:
WITH deduplicated_source AS (
    SELECT 
        s.*,
        ROW_NUMBER() OVER (PARTITION BY product_id ORDER BY seqNum DESC) as row_num
    FROM dev.naval_bronze.products s
)
MERGE INTO dev.naval_silver.product_scd t
USING (SELECT * FROM deduplicated_source WHERE row_num = 1
) s
ON t.product_id = s.product_id
WHEN MATCHED and s.operation='UPDATE' 
    THEN 
        UPDATE SET 
    t.product_name = s.product_name,
    t.product_category = s.product_category,
    t.product_price = s.product_price
WHEN MATCHED AND s.operation='DELETE'
THEN 
DELETE
WHEN NOT MATCHED THEN 
INSERT 
    (product_id, product_name, product_category, product_price) 
VALUES 
    (s.product_id, s.product_name, s.product_category, s.product_price)